<a href="https://colab.research.google.com/github/sarveshsuresh/FairnessTest/blob/main/FairnessFrameworkModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from statsmodels.stats.proportion import proportions_ztest
import pickle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def preprocess(dfr,target):
  df=dfr
  cols = list(df.columns)
  cols.remove(target)
  num_cols = list(df.drop(columns=[target])._get_numeric_data().columns)
  cat_cols=list(set(cols) - set(num_cols))
  #return(num_cols,cat_cols)

  for column in num_cols:
    if len(list(df[column].unique()))<10:
      df[column]=df[column].astype('str')
      df[column].fillna('missing',inplace=True)

    else:
      df[column].fillna(np.mean(df[column]),inplace=True)

  for column in cat_cols:
    df[column].fillna('missing',inplace=True)
    if len(list(df[column].unique()))>10:
      df=df.drop(columns=[column])


  

  return(df)

    



 




In [3]:
def modelling(dfr,target,model,test_size,sens):
  df=dfr
  cols = list(df.columns)
  cols.remove(target)
  num_cols = list(df.drop(columns=[target])._get_numeric_data().columns)
  cat_cols=list(set(cols) - set(num_cols))
  if model =='xgb' or model=='dt' or model=='rf' or model=='lgbm' or model=='catboost':
    for column in cat_cols:
      encoder=LabelEncoder()
      
        

      df[column] = (encoder.fit_transform(df[column]))
      if column==sens:

        sencoder=encoder
        #print('het')
    y=df[target]
    df_train,df_test=train_test_split(df,test_size=test_size,stratify=y) #stratified sampling
    X,Y=df_train.drop(columns=[target]),df_train[target]
    #print(X)

    if model=='dt':
      clf = tree.DecisionTreeClassifier()
      clf = clf.fit(X, Y)
      df_test['prediction']=clf.predict(df_test.drop(columns=[target]))
      data_crosstab = pd.crosstab(df_test[target],
                          df_test['prediction'], 
                              margins = False)
      
      #return(data_crosstab)
      #print(sencoder.inverse_transform(df_test[sens]))
      df_test[sens]=sencoder.inverse_transform(df_test[sens])
      acc=accuracy_score(df_test[target],df_test['prediction'])
      return(df_test,acc)

    if model=='xgb':
      params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

      xgb = XGBClassifier(learning_rate=0.08, n_estimators=500, objective='binary:logistic',
                    silent=True, nthread=1)
      folds = 3
      param_comb = len(params)

      skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)

      random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=4, cv=skf.split(X,Y), random_state=42 )

      random_search.fit(X,Y)

      df_test['prediction']=random_search.predict(df_test.drop(columns=[target]))
      data_crosstab_xgb = pd.crosstab(df_test[target],
                          df_test['prediction'], 
                              margins = False)
      
      #return(data_crosstab_xgb)
      df_test[sens]=sencoder.inverse_transform(df_test[sens])
      acc=accuracy_score(df_test[target],df_test['prediction'])
      return(df_test,acc)

      


  elif model=='logreg' or model=='svm':
    stored_column=df[sens]
    df=pd.get_dummies(df,columns=cat_cols,drop_first=True)
    df[sens]=stored_column
    y=df[target]
    df_train,df_test=train_test_split(df,test_size=test_size,stratify=y) #stratified sampling
    
    for column in num_cols:
      df_train[column]=(df_train[column]-np.mean(df_train[column]))/np.std(df_train[column])
      df_test[column]=(df_test[column]-np.mean(df_train[column]))/np.std(df_train[column])
    X,Y=df_train.drop(columns=[target,sens]),df_train[target]

    if model=='logreg':
      clf=LogisticRegression()
      clf.fit(X,Y)
      df_test['prediction']=clf.predict(df_test.drop(columns=[target,sens]))
      data_crosstab = pd.crosstab(df_test[target],
                          df_test['prediction'], 
                              margins = False)
      
      #return(data_crosstab)

      acc=accuracy_score(df_test[target],df_test['prediction'])
      return(df_test,acc)


    

  #return(df)
  







In [4]:
def prop_parity(cross,tp,fp,tn,fn):
  val=(tp+fp)/(tp+fp+tn+fn)
  return(val)

def equalized_odds(cross,tp,fp,tn,fn):
  val=(tp)/(tp+fn)
  return(val)

def pred_rate_parity(cross,tp,fp,tn,fn):
  val=(tp)/(tp+fp)
  return(val)

def accuracy_parity(cross,tp,fp,tn,fn):
  val=(tp+tn)/(tp+fp+tn+fn)
  return(val)

def false_negrate_parity(cross,tp,fp,tn,fn):
  val=(fn)/(tp+fn)
  return(val)

def false_posrate_parity(cross,tp,fp,tn,fn):
  val=(fp)/(fp+tn)
  return(val)


def neg_predval_parity(cross,tp,fp,tn,fn):
  val=(tn)/(tn+fn)
  return(val)

def specificity_parity(cross,tp,fp,tn,fn):
  val=(tn)/(fp+tn)
  return(val)



In [5]:
def compare_groups(dataframe,column,metric,actual,preds,pos,neg):
  dff=pd.DataFrame()
  levells=[]
  levels=dataframe[column].unique()
  vals=[]
  for level in levels:
    print(level)
    try:
      sub=dataframe[dataframe[column]==level]
      data_crosstab = pd.crosstab(sub[actual],
                              sub[preds], 
                                margins = False)
      print(data_crosstab)
      tp=data_crosstab[pos][pos]
      tn=data_crosstab[neg][neg]
      fp=data_crosstab[pos][neg]
      fn=data_crosstab[neg][pos]
      vals.append(metric(data_crosstab,tp,tn,fp,fn))
      levells.append(level)
    except:
      pass
  dff['level'],dff['metric']=levells,vals
    
  return(dff)

In [8]:
data=pd.read_csv('/content/train_u6lujuX_CVtuZ9i (1).csv')

In [ ]:
data['Gender'].unique()[0]

In [46]:
def compare_groups_multiclass(dataframe,column,metric,reference,actual,preds):
  datafr=dataframe
  dff=pd.DataFrame()
  levells=[]
  levels=datafr[column].unique()
  vals=[]
  if len(levels)>2:
    datafr[actual]=np.where(datafr[actual]==reference,datafr[actual],'not '+reference)
    datafr[preds]=np.where(datafr[preds]==reference,datafr[preds],'not '+reference)
    pos=reference
    neg='not '+reference
    for level in levels:
      print(level)
      try:
        sub=datafr[datafr[column]==level]
        data_crosstab = pd.crosstab(sub[actual],
                                  sub[preds], 
                                    margins = False)
        print(data_crosstab)
        tp=data_crosstab[pos][pos]
        tn=data_crosstab[neg][neg]
        fp=data_crosstab[pos][neg]
        fn=data_crosstab[neg][pos]
        
        vals.append(metric(data_crosstab,tp,tn,fp,fn))
        levells.append(level)
        

      except:
        pass

    dff['level'],dff['metric']=levells,vals
  else:
    pos=reference
    neg=datafr[datafr[actual]!=reference][actual].unique()[0]

    for level in levels:
      try:
        print(level)
        
        sub=datafr[datafr[column]==level]
        data_crosstab = pd.crosstab(sub[actual],
                                  sub[preds], 
                                    margins = False)
        print(data_crosstab)
      
        tp=data_crosstab[pos][pos]
        
        tn=data_crosstab[neg][neg]
        fp=data_crosstab[pos][neg]
        fn=data_crosstab[neg][pos]
        

        vals.append(metric(data_crosstab,tp,tn,fp,fn))
        

        levells.append(level)
      except:
        pass
      

      

    dff['level'],dff['metric']=levells,vals
    print(dff)
  return(dff)



In [6]:
def threshold_eval(tdat,threshold,dat,sens):
  tdat=tdat.sort_values('metric',ascending=False).reset_index(drop=True)
  comp=tdat.loc[0,'metric']
  #tdat['difference']=(comp/tdat['metric'])-1
  #sub=tdat[tdat['difference']>threshold]
  lengths=[]
  pvals=[1]


  for i in range(len(tdat)):
    leng_var=len(dat[dat[sens]==tdat.loc[i,'level']])
    lengths.append(leng_var)
    if i>0:
      leng_fix=len(dat[dat[sens]==tdat.loc[0,'level']])
      
      

      nobs = np.array([leng_fix,leng_var])
      count = np.array([tdat.loc[0,'metric']*leng_fix,tdat.loc[i,'metric']*leng_var])
      stat, pval = proportions_ztest(count, nobs)
      #print((pval))
      pvals.append(pval)
  tdat['Count']=lengths
  tdat['p-value']=pvals
  problems=tdat[tdat['p-value']<threshold]
 


  return(tdat,problems)

In [ ]:
def simple_modelling(df_test,clffile,target,sens):
  clf=pickle.load(open(clffile, "rb"))
  df_test['prediction']=clf.predict(df_test.drop(columns=[target,sens]))
  
  acc=accuracy_score(df_test[target],df_test['prediction'])
  return(df_test,acc)

In [ ]:
def check_fairness(modelfile,testset,target,sens,metric,pos,neg,threshold):
  dataf,acc=simple_modelling(testset,modelfile,target,sens)
  tdata=compare_groups(dataf,sens,metric,target,'prediction',pos,neg)
  if len(tdata)==0:
    Conclusion='Model too simplistic'
  else:
    sum_dat,prob_dat=threshold_eval(tdata,threshold,dataf,sens)
    print('accuracy: ',acc)

    if len(sum_dat)==1:
      Conclusion='Model too simplistic'
    
    elif len(prob_dat)==0:
      Conclusion='The given model is deemed to be fair towards all levels of '+sens+' at the '+str(threshold*100)+' % level of significance'

    else:
      Conclusion='The given model is deemed to be unfair towards the following levels of '+sens+' at the '+str(threshold*100)+' % level of significance : '
      kk=0
      for ii in prob_dat['level']:
        kk+=1
        Conclusion+=ii
        if kk<len(prob_dat):
          Conclusion+=', '




  print(Conclusion)

  lines = ['The Following are the parameters of the fairness-test: ', 'Target variable: '+target,'Model used: '+modelfile,'Sensitive Variable: '+sens,
           'Metric used for calculating parity(fairness): '+metric.__name__,'Level of Significance: '+(str(int(threshold*100)))+' %',Conclusion,'Accuracy of the model: '+str(acc)]
  with open('report.txt', 'w') as f:

    for line in lines:
        f.write(line)
        f.write('\n')


  return(sum_dat)






In [ ]:
def build_model_check_fairness(dfr,target,model,test_size,sens,metric,pos,neg,threshold):
  dat=preprocess(dfr,target)
  dataf,acc=modelling(dat,target,model,test_size,sens)
  tdata=compare_groups(dataf,sens,metric,target,'prediction',pos,neg)
  if len(tdata)==0:
    Conclusion='Model too simplistic'
    sum_dat='No metrics calculated'
  else:
    sum_dat,prob_dat=threshold_eval(tdata,threshold,dataf,sens)
    print('accuracy: ',acc)
    #print(prob_dat)

    

    if len(sum_dat)==1:
      Conclusion='Model too simplistic'
    
    elif len(prob_dat)==0:
      Conclusion='The given model is deemed to be fair towards all levels of '+sens+' at the '+str(threshold*100)+' % level of significance'

    else:
      Conclusion='The given model is deemed to be unfair towards the following levels of '+sens+' at the '+str(threshold*100)+' % level of significance : '
      kk=0
      for ii in prob_dat['level']:
        kk+=1
        Conclusion+=ii
        if kk<len(prob_dat):
          Conclusion+=', '




  print(Conclusion)

  lines = ['The Following are the parameters of the fairness-test: ', 'Target variable: '+target,'Model used: '+model,'Ratio of Test Set Split: '+str(int(test_size*100))+' %','Sensitive Variable: '+sens,
           'Metric used for calculating parity(fairness): '+metric.__name__,'Level of Significance: '+(str(int(threshold*100)))+' %',Conclusion,'Accuracy of the model: '+str(acc)]
  with open('report.txt', 'w') as f:

    for line in lines:
        f.write(line)
        f.write('\n')

  
  return(sum_dat)




In [15]:
def build_model_check_fairness_multiclass(dfr,target,model,test_size,sens,metric,reference,threshold):
  dat=preprocess(dfr,target)
  dataf,acc=modelling(dat,target,model,test_size,sens)
  tdata=compare_groups_multiclass(dataf,sens,metric,reference,target,'prediction')
  if len(tdata)==0:
    Conclusion='Model too simplistic'
    sum_dat='No metrics calculated'
  else:
    sum_dat,prob_dat=threshold_eval(tdata,threshold,dataf,sens)
    print('accuracy: ',acc)
    #print(prob_dat)

    

    if len(sum_dat)==1:
      Conclusion='Model too simplistic'
    
    elif len(prob_dat)==0:
      Conclusion='The given model is deemed to be fair towards all levels of '+sens+' at the '+str(threshold*100)+' % level of significance'

    else:
      Conclusion='The given model is deemed to be unfair towards the following levels of '+sens+' at the '+str(threshold*100)+' % level of significance : '
      kk=0
      for ii in prob_dat['level']:
        kk+=1
        Conclusion+=ii
        if kk<len(prob_dat):
          Conclusion+=', '




  print(Conclusion)

  lines = ['The Following are the parameters of the fairness-test: ', 'Target variable: '+target,'Model used: '+model,'Ratio of Test Set Split: '+str(int(test_size*100))+' %','Sensitive Variable: '+sens,
           'Metric used for calculating parity(fairness): '+metric.__name__,'Level of Significance: '+(str(int(threshold*100)))+' %',Conclusion,'Accuracy of the model: '+str(acc)]
  with open('report.txt', 'w') as f:

    for line in lines:
        f.write(line)
        f.write('\n')

  
  return(sum_dat)


